In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.colors as colors
import pickle
import matplotlib.dates as mdates
import matplotlib
import os

In [ ]:



# Create many demand plots so we can actually see the values
def scrolling_demand(width, region, df, title, save, targets=[-1,], plus=0):
    #plus = 0
    #plus = 225
    #plus = 100
    start = 0 + plus
    end = width+plus
    i = 0
    tot_l = len(df.index)
    while True:
        s = save.replace('.pdf', '_{}cnt_{}plus.pdf'.format(i, plus))
        # Make names ordered by timeslice if we have target slices
        if targets[0] != -1:
            s = s.replace(region, '{}_{}'.format(i, region))
        t = title+': cnt {}'.format(i)
        o = df.loc[start:end]

        #print("scrolling: start {} - end {}".format(start, end))
        # If requested, only plot certain time frames
        if i not in targets and targets[0] != -1:
            if end == tot_l:
                break
            i += 1
            start += width
            end += width
            if end >= tot_l:
                end = tot_l
            continue
            
        # Plot 141 for SOCO
        if 'RAW' in save or 'IMPUTED' in save:
            if i == 141:
                s = save.replace('.pdf', '_{}cnt_{}plus.pdf'.format(i, plus)).replace(region, region)
                # Make names ordered by timeslice if we have target slices
                if targets[0] != -1:
                    s = s.replace(region, '{}_{}'.format(i, region))
                print(start, end, s)
                comparison_demand_plot_clean(o, t, s)
        

        if end == tot_l:
            break
        i += 1
        start += width
        end += width
        if end >= tot_l:
            end = tot_l


    

            

# Shows simplified, clean results     
def comparison_demand_plot_clean(df, title, save):
    plt.close()
    fig, ax = plt.subplots(figsize=(15,7))
    plt.subplots_adjust(bottom=0.2, right=0.94, left=0.14, top=0.96)
    matplotlib.rcParams.update({'font.size': 22})
    #ax.set_xlabel('Hour')
    ax.set_ylabel('Demand (MW)')
    ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
    plt.title('')
    
    if 'RAW' in save:
        ax.plot(df['date_time'], df['raw demand (MW)'], 'k-o', label='Raw Demand')  

    if 'IMPUTED' in save:
        ax.plot(df['date_time'], df['raw demand (MW)'], 'k-o', label='Raw Demand')
        ax.plot(df['date_time'], df['cleaned demand (MW)'], 'mo', label='Imputed Replacement\nValues')
        ax.set_ylim(1207.4499999999998, 62357.55)
    print(ax.get_ylim())
        
    s=9
    show_filters=False
    if show_filters:
        ax.plot(df['date_time'], df['runFiltered'], 'C0o', label='Identical run', markersize=s)
        ax.plot(df['date_time'], df['globalDemandFiltered']*0., 'C5o', label='Global demand', markersize=s)
        ax.plot(df['date_time'], df['globalDemPlusMinusFiltered'], 'C5o', label='_nolegend_', markersize=s)
        ax.plot(df['date_time'], df['deltaFiltered'], 'C2o', label='Double-sided delta', markersize=s)
        ax.plot(df['date_time'], df['deltaSingleFilteredFwd'], 'C3o', label='Single-sided delta', markersize=s)
        ax.plot(df['date_time'], df['deltaSingleFilteredBkw'], 'C3o', label='_nolegend_', markersize=s)
        ax.plot(df['date_time'], df['localDemandFilteredUp'], 'C4o', label='Local demand', markersize=s)
        ax.plot(df['date_time'], df['localDemandFilteredDown'], 'C4o', label='_nolegend_', markersize=s)
        ax.plot(df['date_time'], df['negAndZeroFiltered'], 'C6o', label='Negitive or zero', markersize=s)
        ax.plot(df['date_time'], df['anomalousRegionsFiltered'], 'C1o', label='Anomalous region', markersize=s)
    


    
    plt.legend(prop={'size': 21.5}, ncol=1)
    #ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=30 )
    #ax.xaxis.set_minor_locator(mdates.DayLocator())
    ax.set_ylim(min(ax.get_ylim()[0], 0), ax.get_ylim()[1]*1.5)
    #plt.tight_layout()
    plt.grid()
    plt.savefig(save)
    
    
    
    

In [ ]:
version = '_v1'


png_path = '/Users/truggles/SData_plots{}/'.format(version)
if not os.path.isdir(png_path): os.mkdir(png_path)

region = 'SOCO'


# Plot settings
width = 240
title = '{} Demand Showing Filters'.format(region)
    
targets = [-1,]
plus = 225
plus = 225-19



file_path = '/Users/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Data/data/release_2019_Oct/balancing_authorities//{}.csv'.format(region)
df = pd.read_csv(file_path, na_values=['MISSING', 'EMPTY'])


# Convert date/time
df['date_time'] = pd.to_datetime(df['date_time'])

save = '{}{}_RAW.pdf'.format(png_path, region)
scrolling_demand(width, region, df, title, save, targets, plus)

df['cleaned demand (MW)'] = df['cleaned demand (MW)'].mask(df['cleaned demand (MW)'] == df['raw demand (MW)'])

# Plot initial demand data
save = '{}{}_IMPUTED.pdf'.format(png_path, region)
scrolling_demand(width, region, df, title, save, targets, plus)